In [1]:
import csv
import os
import random
from collections import defaultdict, Counter

csv_file = "game_results.csv"
choices = ["rock", "paper", "scissors"]

def ensure_csv_has_header(csv_file_path, expected_header):
    if not os.path.isfile(csv_file_path) or os.stat(csv_file_path).st_size == 0:
        with open(csv_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(expected_header)

def load_user_choices(csv_file):
    user_choices = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            user_choices.append(row["user_choice"])
    return user_choices

def build_markov_model(user_choices):
    model = defaultdict(Counter)
    for i in range(len(user_choices) - 1):
        current = user_choices[i]
        next_choice = user_choices[i + 1]
        model[current][next_choice] += 1
    return model

def predict_next_move(model, last_choice):
    if last_choice in model and len(model[last_choice]) > 0:
        prediction = model[last_choice].most_common(1)[0][0]
        return prediction
    return random.choice(choices)

def get_counter_move(user_prediction):
    win_map = {
        "rock": "paper",
        "paper": "scissors",
        "scissors": "rock"
    }
    return win_map[user_prediction]

ensure_csv_has_header(csv_file, ["user_choice", "bot_choice", "result"])

user_choices = load_user_choices(csv_file)
markov_model = build_markov_model(user_choices)

with open(csv_file, 'a', newline='') as file:
    writer = csv.writer(file)

    while True:
        user_choice = input("Choose (rock, paper, scissors or exit): ").strip().lower()

        if user_choice == "exit":
            print("Game over.")
            break

        if user_choice not in choices:
            print("Invalid input. Please choose rock, paper, or scissors.")
            continue

        if user_choices:
            last_choice = user_choices[-1]
            predicted_next = predict_next_move(markov_model, last_choice)
            bot_choice = get_counter_move(predicted_next)
        else:
            bot_choice = random.choice(choices)

        print(f"Bot chose: {bot_choice}")

        if user_choice == bot_choice:
            result = "Tie"
            print("It's a tie!")
        elif (
            (user_choice == "rock" and bot_choice == "scissors") or
            (user_choice == "scissors" and bot_choice == "paper") or
            (user_choice == "paper" and bot_choice == "rock")
        ):
            result = "Win"
            print("You win!")
        else:
            result = "Lose"
            print("Bot wins!")

        writer.writerow([user_choice, bot_choice, result])
        user_choices.append(user_choice) 
        markov_model = build_markov_model(user_choices)  

        print("-" * 30)


Choose (rock, paper, scissors or exit):  rock


Bot chose: paper
Bot wins!
------------------------------


Choose (rock, paper, scissors or exit):  rpck


Invalid input. Please choose rock, paper, or scissors.


Choose (rock, paper, scissors or exit):  rock


...

Bot chose: paper
Bot wins!
------------------------------


Choose (rock, paper, scissors or exit):  paper


Bot chose: paper
It's a tie!
------------------------------


Choose (rock, paper, scissors or exit):  exit


Game over.
